# Capetown Housing Prediction Challenge

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#install  necessary packages
# beautiful soup

! pip install beautifulsoup4

In [6]:
#install parser
! pip install lxml

In [7]:
#install requests
! pip install requests


# Web Scraping
Going to scrape the property 24 website for the necessary data to make predictions

In [10]:
# import packages

from bs4 import BeautifulSoup
import requests
import csv

In [11]:
!ls

drive  sample_data


In [13]:
# Create the csv file to save to

info_csv = open('cape_town_housing.csv', 'w')
writer = csv.writer(info_csv)
writer.writerow(['title', 'location', 'bedrooms', 'bathrooms', 'parking_spaces', 'floor_size', 'price'])

67

In [14]:
# scrape propert 24 

for i, value in enumerate(list(range(1,397))):
  url = 'https://www.property24.com/for-sale/cape-town/western-cape/432/p'+ str(value)+ '?PropertyCategory=House%2cApartmentOrFlat%2cTownhouse&sp=pt%3d1000000000000'
  src = requests.get(url).text
  page = BeautifulSoup(src, 'lxml')
  info_box = page.find_all('div', {'class':"p24_regularTile js_rollover_container "}) 

  for value in info_box:
    try:
      price = value.find('span', {'class': "p24_price"}).text
    except Exception as e:
      price = None
    try:
      title = value.find('span', {'class': "p24_title "}).text
    except:
      title = None
    try:
      location = value.find('span', {'class': "p24_location"}).text
    except:
      location = None
    try:
      bedrooms = value.find('span', {'class': "p24_featureDetails", 'title':"Bedrooms"}).text
    except:
      bedrooms = None
    try:
      bathrooms = value.find('span', {'class': "p24_featureDetails", 'title':"Bathrooms"}).text
    except:
      bathrooms = None
    try:
      parking_spaces = value.find('span', {'class': "p24_featureDetails", 'title':"Parking Spaces"}).text
    except Exception as e:
      parking_spaces = None
    try:
      floor_size = value.find('span', {'class': "p24_size", 'title':"Floor Size"}).text
    except:
      floor_size = None

    writer.writerow( [title, location, bedrooms, bathrooms, parking_spaces, floor_size,price])


In [15]:
info_csv.close()

In [16]:
# Read the CSV into Pandas
import pandas as pd
capetown_data = pd.read_csv('cape_town_housing.csv')

In [17]:
#inspect to see if it worked
print(capetown_data.shape)
capetown_data.head()


(7885, 7)


,title,location,bedrooms,bathrooms,parking_spaces,floor_size,price
0,3 Bedroom House,Observatory,3.0,2.0,1.0,NaN,\n\r\n R 3 500 000\r\n
1,1 Bedroom Apartment,De Waterkant,1.0,1.0,NaN,\n\n66 m²\n,\n\r\n R 3 500 000\r\n
2,House,Kirstenhof,NaN,NaN,NaN,NaN,\n\r\n R 1 895 000\r\n
3,4 Bedroom House,Kensington,4.0,1.0,6.0,NaN,\n\r\n R 995 000\r\n
4,House,Rondebosch East,NaN,NaN,NaN,NaN,\r\n POA\r\n


Looks like it worked we scraped 7885 records which will need some cleaning but first let's save to the drive

In [24]:
#save df to drive
!ls

# capetown_data.to_csv('/drive/cape_town_housing.csv')

cape_town_housing.csv  drive  sample_data


In [41]:
# Clean Up Pricing Column

capetown_data['price'] =capetown_data['price'].str.replace('\n','').str.replace('\r','').str.strip().str.replace('R','').str.replace('  ','').str.replace(' ','').str.replace(' ','')

In [47]:
capetown_data['price'] = capetown_data['price'].str.replace('\\xa0','')

  

Some values in the price column are POA, which means property on auction and we cannot use these so we will drop these from the data set 

In [51]:
capetown_data_1 = capetown_data[capetown_data['price'] != 'POA']
capetown_data_1.shape

(7850, 7)

This leaves us with 7850 rows

In [54]:
#convert to price column to integers

capetown_data_1['price'] =capetown_data_1['price'].astype('int')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
superscripts =  "[\u2070\u00b9\u00b2\u00b3\u2074\u2075\u2076\u2077\u2078\u2079]"
superscripts

'[⁰¹²³⁴⁵⁶⁷⁸⁹]'

In [72]:
#Cleaning the floor size column

floor_size = capetown_data_1['floor_size'].str.replace('\n', '').str.replace(' ','').str.strip().str.replace(superscripts,'').str.replace('m','')
floor_size = floor_size.astype(float, errors= 'ignore')

In [73]:
capetown_data_1['floor_size'] = floor_size


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
capetown_data_1.dtypes

title              object
location           object
bedrooms          float64
bathrooms         float64
parking_spaces    float64
floor_size        float64
price               int64
dtype: object

In [75]:
capetown_data_1.head()

,title,location,bedrooms,bathrooms,parking_spaces,floor_size,price
0,3 Bedroom House,Observatory,3.0,2.0,1.0,NaN,3500000
1,1 Bedroom Apartment,De Waterkant,1.0,1.0,NaN,66.0,3500000
2,House,Kirstenhof,NaN,NaN,NaN,NaN,1895000
3,4 Bedroom House,Kensington,4.0,1.0,6.0,NaN,995000
5,2 Bedroom House,Claremont,2.0,1.0,NaN,NaN,2395000


In [76]:
capetown_data_1.title.value_counts(dropna=False)

2 Bedroom Apartment      2112
1 Bedroom Apartment      1430
3 Bedroom House          1056
4 Bedroom House           815
3 Bedroom Apartment       535
2 Bedroom House           369
5 Bedroom House           363
0.5 Bedroom Apartment     208
Apartment                 196
6 Bedroom House           159
3 Bedroom Townhouse       151
2 Bedroom Townhouse        97
4 Bedroom Apartment        81
7 Bedroom House            56
House                      36
1 Bedroom House            33
8 Bedroom House            28
9 Bedroom House            24
4 Bedroom Townhouse        22
10 Bedroom House            7
1.5 Bedroom Apartment       6
2.5 Bedroom Apartment       6
11 Bedroom House            6
12 Bedroom House            5
5 Bedroom Apartment         5
Townhouse                   4
6 Bedroom Apartment         4
13 Bedroom House            4
2.5 Bedroom House           3
5 Bedroom Townhouse         3
15 Bedroom House            2
1 Bedroom Townhouse         2
3.5 Bedroom Apartment       2
16 Bedroom

As we can see above the title column contains the words House, Townhouse or Apartment so we will convert this to those respective types of property

In [78]:
def convert_type(title_name):
  if 'Apartment' in title_name:
    return 'Apartment'
  if 'Townhouse'in title_name:
    return 'Townhouse'
  if 'House' in title_name:
    return 'House'
  return title_name

property_type = capetown_data_1['title'].apply(convert_type)

In [80]:
capetown_data_1['property_type'] = property_type

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
capetown_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7850 entries, 0 to 7884
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           7850 non-null   object 
 1   location        7850 non-null   object 
 2   bedrooms        7614 non-null   float64
 3   bathrooms       7754 non-null   float64
 4   parking_spaces  6358 non-null   float64
 5   floor_size      4441 non-null   float64
 6   price           7850 non-null   int64  
 7   property_type   7850 non-null   object 
dtypes: float64(4), int64(1), object(3)
memory usage: 872.0+ KB


After some basic cleaning we can see that we have some null values in the bedrooms, bathrooms, parking_spaces and floor_size columns. floor_size has the most missing values with nearly 45% of the values missing so our imputation strategy here will be very critcal in coming up with an accurate model

# Modeling

In [83]:
capetown_data_1.head()

,title,location,bedrooms,bathrooms,parking_spaces,floor_size,price,property_type
0,3 Bedroom House,Observatory,3.0,2.0,1.0,NaN,3500000,House
1,1 Bedroom Apartment,De Waterkant,1.0,1.0,NaN,66.0,3500000,Apartment
2,House,Kirstenhof,NaN,NaN,NaN,NaN,1895000,House
3,4 Bedroom House,Kensington,4.0,1.0,6.0,NaN,995000,House
5,2 Bedroom House,Claremont,2.0,1.0,NaN,NaN,2395000,House


We will be dropping the title column from our data set as we have extracted the property type from it

In [84]:
full_data = capetown_data_1.drop('title', axis=1)
full_data.head()

,location,bedrooms,bathrooms,parking_spaces,floor_size,price,property_type
0,Observatory,3.0,2.0,1.0,NaN,3500000,House
1,De Waterkant,1.0,1.0,NaN,66.0,3500000,Apartment
2,Kirstenhof,NaN,NaN,NaN,NaN,1895000,House
3,Kensington,4.0,1.0,6.0,NaN,995000,House
5,Claremont,2.0,1.0,NaN,NaN,2395000,House


In [86]:
#Break Off X and y
X= full_data.drop('price', axis = 1)
y =full_data['price']

In [87]:
# One Hot Encode X categorical variables

X_full = pd.get_dummies(X)


### Comparing Different Imputation Strategies Using Random Forrest Regressor

In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [89]:
# Splitting the data into train and validation sets

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

###### Testing for dropping all row with null values from the dataset

In [90]:
# get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                    if X_train[col].isnull().any()] 

# drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis = 1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis = 1)

In [91]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
2777949.4160192064


###### Testing for Mean Value Imputation for missing columns



In [92]:
from sklearn.impute import SimpleImputer

# imputation
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# imputation removed column names putting them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [94]:
print("MAE Mean value Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE Mean value Imputation):
1717188.00888379


###### Testing for Modal Value Imputation for missing columns

In [96]:
# imputation
imputer = SimpleImputer(strategy= 'most_frequent')
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# imputation removed column names putting them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [97]:
print("MAE Most Frequent value Imputation:")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE Most Frequent value Imputation:
1757090.2026886295


It appears that the best method of imputation for our dataset as a whole is mean value imputation

# Trying Different Models

#### XGBoostRegressor

In [99]:
# Rerun Mean Imputation

# imputation
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# imputation removed column names putting them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [100]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor() 

# Fit the model
my_model_1.fit(imputed_X_train, y_train) 

[05:31:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [101]:
# make predictions
# Get predictions
predictions_1 = my_model_1.predict(X_valid) 

In [102]:
# Calculate MAE
mae_1 = mean_absolute_error(y_valid, predictions_1) 

print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 2190489.554219745


In [104]:
# Let's try tune the model somewhat

# Define the model
my_model_2 = XGBRegressor(n_estimators=2000, learning_rate = 0.05, n_jobs = 10) 

# Fit the model
my_model_2.fit(imputed_X_train, y_train,
              eval_set = [(imputed_X_valid, y_valid)],
              verbose=100) 

# Get predictions
predictions_2 = my_model_2.predict(X_valid) 

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid) 

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

[05:37:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:9.71306e+06
[100]	validation_0-rmse:5.12617e+06
[200]	validation_0-rmse:4.86178e+06
[300]	validation_0-rmse:4.7489e+06
[400]	validation_0-rmse:4.69211e+06
[500]	validation_0-rmse:4.65796e+06
[600]	validation_0-rmse:4.60752e+06
[700]	validation_0-rmse:4.57706e+06
[800]	validation_0-rmse:4.53435e+06
[900]	validation_0-rmse:4.51745e+06
[1000]	validation_0-rmse:4.51073e+06
[1100]	validation_0-rmse:4.50705e+06
[1200]	validation_0-rmse:4.5015e+06
[1300]	validation_0-rmse:4.4892e+06
[1400]	validation_0-rmse:4.49085e+06
[1500]	validation_0-rmse:4.49244e+06
[1600]	validation_0-rmse:4.48535e+06
[1700]	validation_0-rmse:4.48461e+06
[1800]	validation_0-rmse:4.48135e+06
[1900]	validation_0-rmse:4.49521e+06
[1999]	validation_0-rmse:4.49361e+06
Mean Absolute Error: 1825659.825347084
